In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input
from sklearn.preprocessing import StandardScaler

# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
print("GPUs available:", gpus)

if gpus:
    try:
        # (Optional) set memory growth so TF doesn’t grab all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPUs.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected, running on CPU.")


In [ ]:
# Start working with TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local') # Detect TPU
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
# Load and preprocess data with day_type
combinations_df = pd.read_csv(
    '../input/line321_combinations_exp.csv',
    usecols=[
        'to_station',
        'current_stop_index',
        'current_delay',
        'target_stop_index',
        'target_delay',
        'day_type'
    ],
    dtype={
        'to_station': 'bool',
        'current_stop_index': np.int8,
        'current_delay': np.float16,
        'target_stop_index': np.int8,
        'target_delay': np.float16,
        'day_type': np.int8
    }
)

X_raw = combinations_df[['to_station', 'current_stop_index', 'current_delay',
                         'target_stop_index', 'day_type']]
y = combinations_df['target_delay']

# Experimental day_type and WEATHER
# combinations_df = pd.read_csv('../input/line321_combinations_with_weather.csv', usecols=['to_station', 'current_stop_index', 'current_delay', 'target_stop_index', 'target_delay', 'day_type', 'FR/windspeed (0.1 m/s)', 'TG/temperature (0.1 °C)', 'RH/precipitation (0.1mm)'], dtype={'to_station': 'bool', 'current_stop_index': np.int8, 'current_delay': np.float16, 'target_stop_index': np.int8, 'target_delay': np.float16, 'day_type': np.int8, 'FR/windspeed (0.1 m/s)': np.float16, 'TG/temperature (0.1 °C)': np.float16, 'RH/precipitation (0.1mm)': np.float16})
# X_raw = combinations_df[['to_station', 'current_stop_index', 'current_delay', 'target_stop_index', 'day_type', 'FR/windspeed (0.1 m/s)', 'TG/temperature (0.1 °C)', 'RH/precipitation (0.1mm)']]
# y = combinations_df['target_delay']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

# Convert to float32 for fast GPU ops
X_scaled = X_scaled.astype('float32')
y = y.values.astype('float32')

# Reshape for LSTM: [samples, timesteps, features]
# Each sample = 1 timestep (tabular -> sequence length 1)
X = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Train/test split (we'll create a validation split in the next cell)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:

# Check GPU
print("TF version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))

# Optional: avoid TF grabbing all GPU memory at once
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPUs.")
    except RuntimeError as e:
        print(e)

# Create explicit train/validation split from training data
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

batch_size = 256  # Larger batch to better use the GPU

# Build tf.data pipelines for efficient feeding
train_ds = (
    tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
    .shuffle(len(X_tr))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# Build LSTM model (cuDNN-friendly settings)
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(
        256,                        # a bit larger to benefit from GPU
        return_sequences=False,
        activation='tanh',
        recurrent_activation='sigmoid',
        recurrent_dropout=0.0,      # must be 0.0 for cuDNN fast path
        unroll=False,
        use_bias=True,
        unit_forget_bias=True
    ),
    Dropout(0.2),
    Dense(48, activation='relu'),
    Dense(1)
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Train model
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# Evaluate model
y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Calculate NRMSE (Normalized RMSE, normalized by the range of y_test)
nrmse = rmse / (np.max(y_test) - np.min(y_test))

# Calculate SMAPE (Symmetric Mean Absolute Percentage Error)
smape = 100 * np.mean(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Calculate 90th Percentile Absolute Error (P90 AE)
p90_ae = np.percentile(np.abs(y_test - y_pred), 90)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R^2 Score: {r2:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Symmetric MAPE (SMAPE): {smape:.2f}%")
print(f"Normalized RMSE (NRMSE): {nrmse:.4f}")
print(f"90th Percentile Absolute Error (P90 AE): {p90_ae:.2f}")


## LSTM Hyperparameter Optimization with KerasTuner
This cell uses KerasTuner to search for the best LSTM architecture and training parameters to minimize MAE.

In [ ]:
import keras_tuner as kt

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import keras_tuner as kt
from sklearn.model_selection import train_test_split

# Ensure float32 (good for GPU + cuDNN)
X_train = X_train.astype("float32")
y_train = y_train.astype("float32")

# Fixed train/val split used for ALL tuner trials
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

def build_lstm_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    # 4 choices → 32, 64, 96, 128
    model.add(
        LSTM(
            # units=hp.Choice("lstm_units", [32, 64, 96, 128]),
            units=hp.Choice("lstm_units", [32, 64, 96]),
            return_sequences=False,
            activation="tanh",
            recurrent_activation="sigmoid",
            recurrent_dropout=0.0,   # keep cuDNN fast path
            unroll=False,
            use_bias=True,
            unit_forget_bias=True
        )
    )

    # 5 choices → 0.1, 0.2, 0.3, 0.4, 0.5
    model.add(
        Dropout(
            rate=hp.Choice("dropout", [0.2, 0.3, 0.4])
        )
    )

    # 4 choices → 16, 32, 48, 64
    # model.add(
    #     Dense(
    #         units=hp.Choice("dense_units", [16, 32, 48, 64]),
    #         units=hp.Choice("dense_units", [32]),
    #         activation="relu"
    #     )
    # )
    model.add(Dense(32, activation="relu"))

    model.add(Dense(1))

    # 5 choices → total 400 combos
    lr = hp.Choice(
        "learning_rate",
        # values=[1e-4, 3e-4, 1e-3, 3e-3, 1e-2]
        values=[1e-4, 5e-4, 1e-3]
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [ ]:
import keras_tuner as kt

tuner = kt.BayesianOptimization(
    hypermodel=build_lstm_model,
    objective="val_mae",
    max_trials=20,               # number of configurations to try
    num_initial_points=5,        # random warmup before it gets "smart"
    directory="./data/lstm_bayes",
    project_name="bayes_321",
    overwrite=True
)

tuner.search_space_summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

tuner.search(
    X_tr, y_tr,
    epochs=18,                # upper bound per trial; EarlyStopping will often stop earlier
    batch_size=256,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

tuner.results_summary()

best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:", best_hps.values)


In [ ]:
# Retrieve and train the best model

best_hp = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.hypermodel.build(best_hp)
history = best_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
    verbose=1
)

# Evaluate on test set
y_pred = best_model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Tuned LSTM - MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}")

In [ ]:

# Example predictions: Glaspoort to Piazza and Evoluon to Piazza
X_new = np.zeros((1, 1, X_train.shape[2]))
col_idx = {col: i for i, col in enumerate(X_encoded.columns)}
X_new[0, 0, col_idx['current_stop_index']] = 13
X_new[0, 0, col_idx['current_delay']] = -60
X_new[0, 0, col_idx['target_stop_index']] = 15
X_new[0, 0, col_idx['to_station']] = 1
# Set day_type columns as needed (e.g., X_new[0, 0, col_idx['day_type_Monday']] = 1)
y_new_pred = model.predict(X_new).flatten()
print(f'Glaspoort to Piazza delay (LSTM): {y_new_pred[0]:.2f}')

X_evoluon = np.zeros((1, 1, X_train.shape[2]))
X_evoluon[0, 0, col_idx['current_stop_index']] = 10
X_evoluon[0, 0, col_idx['current_delay']] = 120
X_evoluon[0, 0, col_idx['target_stop_index']] = 15
X_evoluon[0, 0, col_idx['to_station']] = 1
# Set day_type columns as needed
y_evoluon_pred = model.predict(X_evoluon).flatten()
print(f'Evoluon to Piazza delay (LSTM): {y_evoluon_pred[0]:.2f}')

In [ ]:
# Save model
model.save('./data/models/lstm_model_L321.keras')

In [ ]:
# Save model
best_model.save('./data/models/lstm_model_L321_optimized.keras')

In [ ]:
# Load model
model = tf.keras.models.load_model('./data/models/lstm_model_L321.h5')

In [ ]:
# Evaluate model
y_pred = best_model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R^2 Score: {r2:.2f}")
